In [1]:
import requests
import tabula
import pandas as pd

# Download of Module Handbooks

In [4]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

Downloaded bachelor_business_management_and_economics.pdf
Downloaded bachelor_information_systems.pdf
Downloaded bachelor_economathematics.pdf
Downloaded master_information_systems.pdf
Downloaded master_management.pdf
Downloaded master_international_economic_policy.pdf


# Reading Tables from Module Book

In [11]:
tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-250', multiple_tables=True)
# Convert each table to a DataFrame

dataframes = [pd.DataFrame(table) for table in tables_master_is]

# You can access each DataFrame using dataframes[index]
# For example, to access the first DataFrame:
df = dataframes[20]
print(df)


                                         Module title       Abbreviation
0         Linear Algebra for Economics and Management  10-M-MWW2-212-m01
1                Module coordinator Module offered by                NaN
2   Dean of Studies Mathematik (Mathematics) Insti...                NaN
3   ECTS Method of grading Only after succ. compl....                NaN
4                                5 numerical grade --                NaN
5           Duration Module level Other prerequisites                NaN
6                         1 semester undergraduate --                NaN
7                                            Contents                NaN
8   Theory of real-valued functions in several var...                NaN
9                          Intended learning outcomes                NaN
10  The student deepens his/her knowledge in analy...                NaN
11  se methods to simple problems in economical mo...                NaN
12  Courses (type, number of weekly contact hours,.

In [12]:
import pandas as pd
import tabula

# Read the PDF and convert each table to a DataFrame
tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-250', multiple_tables=True)
dataframes = [pd.DataFrame(table) for table in tables_master_is]

# Combine all the DataFrames into one overall DataFrame
overall_df = pd.concat(dataframes)

# Print the overall DataFrame
print(overall_df)

In [10]:
print(tables)
df = dataframes[200]
print(df.shape)

[                section / sub-section ECTS credits  starting\rpage
0                  Compulsory Courses          105               9
1                Business Informatics           20              10
2   Business Management and Economics           30              19
3                            Methoden           20              34
4                    Computer Science           35              41
5                Compulsory Electives           45              50
6                Business Informatics      min. 20              51
7             Business Administration      max. 25              83
8                           Economics      max. 25             155
9                    Computer Science      max. 25             230
10                    Key Skills Area           20             242
11                 General Key Skills            5             243
12        Subject-specific Key Skills           15             244
13                        Thesis Area           10           